### Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

overview of project:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
### Open AI API key and Opensource models- LLama3, Gemma2, mistral--Groq
import os
from dotenv import load_dotenv
load_dotenv()


import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022F1D73A690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022F1D875E50>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result = model.invoke(messages)
result

AIMessage(content='**Hello** - Bonjour\n\n**How are you?** - Comment allez-vous? \n\n(More informal: Comment vas-tu?) \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 21, 'total_tokens': 54, 'completion_time': 0.06, 'prompt_time': 0.00132278, 'queue_time': 0.187726133, 'total_time': 0.06132278}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7487e8e-0054-4925-a0f9-492037ecfb09-0', usage_metadata={'input_tokens': 21, 'output_tokens': 33, 'total_tokens': 54})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to greet someone.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is a more casual way to greet someone you know well.)\n* **Coucou, comment ça va ?** (This is a very informal way to greet someone, similar to "Hey, how\'s it going?")\n\n\nLet me know if you\'d like to learn more French greetings! \n'

In [8]:
### Using LCEL- chain the components
chain = model | parser
chain.invoke(messages)


'Hello: Bonjour \n\nHow are you?: Comment allez-vous? (formal) or Ça va? (informal) \n\n\nSo, together they would be:\n\n**Bonjour, comment allez-vous?** (formal)\n**Bonjour, ça va?** (informal) \n'

In [9]:
## Prompt Template
from langchain_core.prompts import ChatPromptTemplate
genric_template = "Translate the following into {Language}"
prompt = ChatPromptTemplate.from_messages(
    [("system", genric_template), ("user", "{text}")]
)

prompt

ChatPromptTemplate(input_variables=['Language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Language'], input_types={}, partial_variables={}, template='Translate the following into {Language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [11]:
result = prompt.invoke({
    "Language": "French",
    "text": "Hello",
})

result

In [14]:
chain = prompt | model | parser
chain.invoke({"Language": "French", "text": "Hello"})

'Bonjour \n'